In [1]:
from pl_modules import HuggingfaceDatasetWrapper, PlAutoModelForSequenceClassification, HuggingfaceTrainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
text_classificator = PlAutoModelForSequenceClassification(model_name_or_path='distilbert-base-cased')

AssertionError: failed to inspect the obj init

In [3]:
from inspect import getfullargspec
getfullargspec(AutoModelForSequenceClassification.from_pretrained)

FullArgSpec(args=['cls', 'pretrained_model_name_or_path'], varargs='model_args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [ ]:
from transformers import AutoModelForSequenceClassification
type(AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased'))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification